In [1]:
import plotly
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import pandas

from util import ESConnection
from elasticsearch_dsl import Search, Q

es_conn = ESConnection()

init_notebook_mode(connected=True)


INIT_DATE_4Y = '2013-10-01'
INIT_DATE_1Y = '2016-10-01'
END_DATE = '2017-10-01'

## Basic metrics to be in the report:

**Git**
* Evolution and trends over time [per quarter] of commits by gender
  * Commits by gender (columns: hash, gender)
* Evolution and trends [per quarter] of developers over time by gender
  * Developers by gender (columns: name, uuid, gender)
* Evolution and trends of type of contributions (code or others) by gender over time
  * Type of file touched by developers (columns: filetype, gender)

**Gerrit**
* Evolution of code reviews over time by gender
  * Count votes by gender (vote, gender)
* Evolution of code reviews developers over time by gender
  * Count people voting (name, uuid, vote)
* Evolution of core reviews over time by gender
  * Votes +2 or -2 (vote +2/-2 and gender)
* Evolution of core review developers over time by gender
  * People voting +2 or -2 (name, uuid, vote +2/-2, gender)

**Others**
* Evolution of attracted developers over time by gender
  * First commit by gender
* Time working in the community by gender
  * Time difference between the first and last contribution by all developers (so how long developers remain in OpenStack?).

Some Functions
========

In [2]:
def query_metric_over_time(index, metric_name, metric_field, filters = []):
    
    s = Search(using=es_conn, index=index)  # Index selection
    for filtering in filters:
        s = s.filter(filtering)
    s.aggs.bucket('gender', 'terms', field='gender')\
          .bucket('time', 'date_histogram', field='date', interval='quarter')\
          .metric(metric_name, 'cardinality', field=metric_field, precision_threshold=10000)
    result = s.execute()

    value = result.to_dict()["aggregations"]['gender']['buckets']
    
    df = pandas.DataFrame()
    for i in value:
        df2 = (pandas.DataFrame.from_dict(i["time"]["buckets"]))
        df2["gender"] = i["key"]
        df2[metric_name] = df2[metric_name].apply(lambda row:row["value"])
        df = pandas.concat([df, df2])
        
    return df

In [3]:
def query_total_piechart(index, metric_name, metric_field, filters = []):
    s = Search(using=es_conn, index=index)  # Index selection
    for filtering in filters:
        s = s.filter(filtering)
    #s = s.filter('range', date={'gt': start_date, 'lt':'now/M'}) # filter date
    s.aggs.bucket('gender', 'terms', field='gender')\
          .metric(metric_name, 'cardinality', field=metric_field, precision_threshold=1000000)
    result = s.execute()
    
    buckets = result.to_dict()["aggregations"]["gender"]["buckets"]
    pie_chart_labels = []
    pie_chart_values = []
    pie_chart_count = []
    for bucket in buckets:
        pie_chart_labels.append(bucket["key"])
        pie_chart_values.append(bucket[metric_name]["value"])
        pie_chart_count.append(bucket["doc_count"])
        
    return pie_chart_labels, pie_chart_values, pie_chart_count

In [4]:
def draw_evolutionary_chart(label, values):
    female_changesets = plotly.graph_objs.Scatter(
    x = df[df["gender"]=="female"][label],
    y = df[df["gender"]=="female"][values],
    name = 'female'
    )

    male_changesets = plotly.graph_objs.Scatter(
        x = df[df["gender"]=="male"][label],
        y = df[df["gender"]=="male"][values],
        name = 'male'
    )

    unknown_changesets = plotly.graph_objs.Scatter(
        x = df[df["gender"]=="NotKnown"][label],
        y = df[df["gender"]=="NotKnown"][values],
        name = 'unknown'
    )

    iplot([female_changesets, male_changesets, unknown_changesets])
    # Using only plot and not iplot creates and HTML page with JS to play with the chart

In [5]:
def draw_piechart(pie_chart_labels, pie_chart_values, title):
            
    fig = {
    'data': [{'labels': pie_chart_labels,
              'values': pie_chart_values,
              'type': 'pie'}],
    'layout': {'title': title}
     }
    iplot(fig)

Gerrit
=====

In [8]:
INDEX = "gerrit_openstack_gender_uuids"
filter_date_4y = Q('range', date={'gte': INIT_DATE_4Y, 'lt': END_DATE})
filter_date_1y = Q('range', date={'gte': INIT_DATE_1Y, 'lt': END_DATE})

## Changeset Submissions by Gender

### Evolution of submissions sent over time by gender

In [13]:
METRIC_NAME = "changesets"
METRIC_FIELD = "id"
filter_changeset_submission = Q('term', eventtype='CHANGESET_SENT') # filter by event: vote a code review
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, filter_changeset_submission])

In [14]:
draw_evolutionary_chart("key_as_string", METRIC_NAME)

### Aggregated changeset submissions by gender

In [15]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y,
                                                                                             filter_changeset_submission])

In [16]:
draw_piechart(pie_chart_labels, pie_chart_values, 'Changeset Submissions by Gender (last 4 years)')

In [17]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_1y,
                                                                                             filter_changeset_submission])


In [18]:
draw_piechart(pie_chart_labels, pie_chart_values, 'Changeset Submissions by Gender (last year)')

## Population of people submitting changesets
### Evolution of submitters over time by gender
  * Count people submitting (id, uuid)

In [19]:
METRIC_NAME = "submitters"
METRIC_FIELD = "owner"
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, filter_changeset_submission])
draw_evolutionary_chart("key_as_string", METRIC_NAME)

### Aggregated number of submitters


In [20]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y,
                                                                                             filter_changeset_submission])
draw_piechart(pie_chart_labels, pie_chart_values, 'Changeset Submitters by Gender (last 4 years)')

In [21]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD,  [filter_date_1y,
                                                                                              filter_changeset_submission])
draw_piechart(pie_chart_labels, pie_chart_values, 'Changeset Submitters by Gender (last year)')

* Evolution of code reviews developers over time by gender
  * Count people voting (name, uuid, vote)


## Number of votes by gender

In [22]:
METRIC_NAME = "reviewer"
METRIC_FIELD = "owner"
filter_vote = Q('term', eventtype='CHANGESET_PATCHSET_APPROVAL_Code-Review') # filter by event: vote a code review

df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, filter_vote])
draw_evolutionary_chart("key_as_string", "doc_count")

In [23]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD,  [filter_date_4y,
                                                                                              filter_vote])
draw_piechart(pie_chart_labels, pie_chart_count, 'Changeset votes by Gender (last 4 years)')

In [24]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD,  [filter_date_1y,
                                                                                              filter_vote])
draw_piechart(pie_chart_labels, pie_chart_count, 'Changeset votes by Gender (last year)')

## Number of people voting

In [25]:
draw_evolutionary_chart("key_as_string", METRIC_NAME)

In [26]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD,  [filter_date_4y,
                                                                                              filter_vote])
draw_piechart(pie_chart_labels, pie_chart_values, 'People voting by Gender (last 4 years)')

In [27]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD,  [filter_date_1y,
                                                                                              filter_vote])
draw_piechart(pie_chart_labels, pie_chart_values, 'People voting by Gender (last year)')

## Number of core reviews (-2 OR +2) by gender

In [28]:
METRIC_NAME = "core_reviewers"
METRIC_FIELD = "owner"

filter_core_vote = Q('terms', value=["2", "-2"])
filter_vote = Q('term', eventtype='CHANGESET_PATCHSET_APPROVAL_Code-Review') # filter by event: vote a code review
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_core_vote, filter_date_4y, filter_vote])
draw_evolutionary_chart("key_as_string", "doc_count")

In [29]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD,  [filter_date_4y,
                                                                                              filter_vote, filter_core_vote])
draw_piechart(pie_chart_labels, pie_chart_count, 'Core Reviews by Gender (last 4 years)')

In [30]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD,  [filter_date_1y,
                                                                                              filter_vote, filter_core_vote])
draw_piechart(pie_chart_labels, pie_chart_count, 'Core Reviews by Gender (last year)')

## Number of people acting as core reviewers (-2 OR +2) by gender

In [31]:
draw_evolutionary_chart("key_as_string", METRIC_NAME)

In [32]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD,  [filter_date_4y,
                                                                                              filter_vote, filter_core_vote])
draw_piechart(pie_chart_labels, pie_chart_values, 'Core Reviewers by Gender (last 4 years)')

In [33]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD,  [filter_date_1y,
                                                                                              filter_vote, filter_core_vote])
draw_piechart(pie_chart_labels, pie_chart_values, 'Core Reviewers by Gender (last year)')

# GIT

In [34]:
INDEX = "git_openstack_gender_uuids_fielddata"
#filter_merges_addedlines = Q('range', addedlines={'gt': 0})
filter_merges_addedlines = Q('range', num_added_lines={'gt': 0})
#filter_merges_removedlines = Q('range', removedlines={'gt': 0})
filter_merges_removedlines = Q('range', num_removed_lines={'gt': 0})
filter_bots = Q('bool', must_not=[Q('match', gender_analyzed_name='Jenkins')])

In [35]:
git_filter = Q('bool',
    must_not=[Q('match', gender_analyzed_name='Jenkins')],
    should=[filter_merges_addedlines, filter_merges_removedlines]
)

* Evolution and trends over time [per quarter] of commits by gender
  * Commits by gender (columns: hash, gender)

In [36]:
METRIC_NAME = "commits"
METRIC_FIELD = "id"
#df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, filter_merges_addedlines,
#                                                               filter_merges_removedlines, filter_bots])
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, git_filter])

In [37]:
draw_evolutionary_chart("key_as_string", METRIC_NAME)

In [38]:
#pie_chart_labels, pie_chart_values = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
#                                                            [filter_date_4y, filter_merges_addedlines,
#                                                             filter_merges_removedlines, filter_bots])

pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_4y, git_filter])

draw_piechart(pie_chart_labels, pie_chart_values, 'Commits by Gender (last 4 years)')

In [39]:
#pie_chart_labels, pie_chart_values = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
#                                                            [filter_date_1y, filter_merges_addedlines,
#                                                             filter_merges_removedlines, filter_bots])

pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_1y, git_filter])

draw_piechart(pie_chart_labels, pie_chart_values, 'Commits by Gender (last year)')


* Evolution and trends [per quarter] of developers over time by gender
  * Developers by gender (columns: name, uuid, gender)

In [40]:
METRIC_NAME = "authors"
METRIC_FIELD = "uuid"

df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, git_filter])

draw_evolutionary_chart("key_as_string", METRIC_NAME)

In [41]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_4y, git_filter])
draw_piechart(pie_chart_labels, pie_chart_values, 'Authors by Gender (last 4 years)')

In [42]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_1y, git_filter])
draw_piechart(pie_chart_labels, pie_chart_values, 'Authors by Gender (last year)')

# Top Projects by women activity and women population

In [43]:
METRIC_NAME = "commits"
METRIC_FIELD = "id"

filters = [filter_date_4y, git_filter]

s = Search(using=es_conn, index=INDEX)  # Index selection
for filtering in filters:
    s = s.filter(filtering)
s.aggs.bucket('project', 'terms', field='projects', size=1000)\
      .bucket('gender', 'terms', field='gender')\
      .metric(METRIC_NAME, 'cardinality', field=METRIC_FIELD, precision_threshold=10000)\
      .metric('authors', 'cardinality', field='uuid')
result = s.execute()

values = result.to_dict()

In [44]:
value = result.to_dict()["aggregations"]['project']['buckets']
print(value)
df = pandas.DataFrame()
df_summary = pandas.DataFrame(columns=["project", "commits", "authors", "ratio_commits", "ratio_authors"])
for i in value:
    df2 = (pandas.DataFrame.from_dict(i["gender"]["buckets"]))
    df2["project"] = i["key"]
    
    df2[METRIC_NAME] = df2[METRIC_NAME].apply(lambda row:row["value"])
    df2['authors'] = df2['authors'].apply(lambda row:row['value'])
    df = pandas.concat([df, df2])
    
    df_summary2 = pandas.DataFrame(columns=["ratio_authors", "ratio_commits", "authors", "commits", "project"])

    df_summary2["project"] = [i["key"]]
    df_summary2["commits"] = [int(df2[df2["key"]=="female"]["commits"])]
    df_summary2["authors"] = [int(df2[df2["key"]=="female"]["authors"])]
    df_summary2["ratio_commits"] = [float(df2[df2["key"]=="female"]["commits"]) / float(df2[df2["key"]=="male"]["commits"])*100]
    df_summary2["ratio_authors"] = [float(df2[df2["key"]=="female"]["authors"]) / float(df2[df2["key"]=="male"]["authors"])*100]


    df_summary = pandas.concat([df_summary, df_summary2])

df_summary = df_summary[["project", "authors", "commits", "ratio_authors", "ratio_commits"]]
#df_summary.columns = ["project", "authors", "commits", "ratio_authors", "ratio_commits"]
    

[{'gender': {'sum_other_doc_count': 0, 'buckets': [{'commits': {'value': 129770}, 'key': 'male', 'authors': {'value': 3981}, 'doc_count': 140299}, {'commits': {'value': 45003}, 'key': 'NotKnown', 'authors': {'value': 1757}, 'doc_count': 49149}, {'commits': {'value': 11950}, 'key': 'female', 'authors': {'value': 587}, 'doc_count': 13088}], 'doc_count_error_upper_bound': 0}, 'key': 'Packaging-deb', 'doc_count': 202536}, {'gender': {'sum_other_doc_count': 0, 'buckets': [{'commits': {'value': 45262}, 'key': 'male', 'authors': {'value': 1229}, 'doc_count': 46607}, {'commits': {'value': 3100}, 'key': 'NotKnown', 'authors': {'value': 283}, 'doc_count': 3293}, {'commits': {'value': 2638}, 'key': 'female', 'authors': {'value': 145}, 'doc_count': 2813}], 'doc_count_error_upper_bound': 0}, 'key': 'Infrastructure', 'doc_count': 52713}, {'gender': {'sum_other_doc_count': 0, 'buckets': [{'commits': {'value': 28813}, 'key': 'male', 'authors': {'value': 888}, 'doc_count': 30536}, {'commits': {'value':

TypeError: cannot convert the series to <class 'int'>

In [45]:
print(df_summary)

   authors  commits             project  ratio_authors  ratio_commits
0    587.0  11950.0       Packaging-deb      14.745039       9.208600
0    145.0   2638.0      Infrastructure      11.798210       5.828289
0    149.0   4150.0        notavailable      16.779279      14.403221
0      8.0    637.0    OpenStack Charms       7.339450       3.642498
0     89.0    706.0             neutron      16.036036       6.961838
0     26.0    350.0    OpenStackAnsible      14.942529       2.972147
0     97.0    766.0                nova      16.193656       7.432564
0    119.0   2266.0       Documentation      21.915285      33.382440
0     35.0    596.0             tripleo      12.681159       5.052132
0     32.0    868.0    Puppet OpenStack       9.638554      10.886743
0     89.0    577.0   Quality Assurance      14.037855       8.326118
0     43.0   1341.0              ironic      14.878893      25.131184
0     52.0    244.0                oslo      13.131313       4.675225
0     51.0    206.0 

In [46]:
print(df_summary.sort_values("ratio_commits", ascending=False))

   authors  commits             project  ratio_authors  ratio_commits
0      8.0     44.0           tricircle      66.666667     115.789474
0     18.0    201.0              senlin      35.294118      64.012739
0     13.0    132.0                 zun      39.393939      51.162791
0    119.0   2266.0       Documentation      21.915285      33.382440
0     12.0    198.0             vitrage      46.153846      30.045524
0     35.0    844.0              murano      23.489933      29.739253
0     10.0    109.0               kuryr      20.408163      29.066667
0     43.0   1341.0              ironic      14.878893      25.131184
0     10.0    251.0            congress      12.987013      25.000000
0     13.0    237.0          dragonflow      33.333333      24.687500
0     83.0    780.0             horizon      23.446328      23.207379
0     27.0    334.0              magnum      18.750000      22.767553
0     37.0    386.0     OpenStackClient      20.218579      21.035422
0     19.0    173.0 

In [47]:
df_summary.sort_values("ratio_authors", ascending=False)


,authors,commits,project,ratio_authors,ratio_commits
0,8.0,44.0,tricircle,66.666667,115.789474
0,12.0,198.0,vitrage,46.153846,30.045524
0,13.0,132.0,zun,39.393939,51.162791
0,8.0,12.0,karbor,36.363636,5.970149
0,18.0,201.0,senlin,35.294118,64.012739
0,13.0,237.0,dragonflow,33.333333,24.687500
0,27.0,119.0,Security,24.107143,7.787958
0,28.0,152.0,tacker,23.728814,20.512821
0,35.0,844.0,murano,23.489933,29.739253
0,83.0,780.0,horizon,23.446328,23.207379


In [48]:
print(df_summary.sort_values("authors", ascending=False))


   authors  commits             project  ratio_authors  ratio_commits
0    587.0  11950.0       Packaging-deb      14.745039       9.208600
0    149.0   4150.0        notavailable      16.779279      14.403221
0    145.0   2638.0      Infrastructure      11.798210       5.828289
0    119.0   2266.0       Documentation      21.915285      33.382440
0     97.0    766.0                nova      16.193656       7.432564
0     89.0    577.0   Quality Assurance      14.037855       8.326118
0     89.0    706.0             neutron      16.036036       6.961838
0     83.0    780.0             horizon      23.446328      23.207379
0     69.0    489.0              cinder      14.838710      10.264484
0     52.0    244.0                oslo      13.131313       4.675225
0     52.0    572.0            keystone      18.055556      10.711610
0     51.0    206.0                heat      16.398714       3.711712
0     51.0    224.0           Telemetry      22.666667       7.598372
0     49.0    217.0 

In [49]:
print(df_summary.sort_values("commits", ascending=False))


   authors  commits             project  ratio_authors  ratio_commits
0    587.0  11950.0       Packaging-deb      14.745039       9.208600
0    149.0   4150.0        notavailable      16.779279      14.403221
0    145.0   2638.0      Infrastructure      11.798210       5.828289
0    119.0   2266.0       Documentation      21.915285      33.382440
0     43.0   1341.0              ironic      14.878893      25.131184
0     32.0    868.0    Puppet OpenStack       9.638554      10.886743
0     35.0    844.0              murano      23.489933      29.739253
0     83.0    780.0             horizon      23.446328      23.207379
0     97.0    766.0                nova      16.193656       7.432564
0     89.0    706.0             neutron      16.036036       6.961838
0      8.0    637.0    OpenStack Charms       7.339450       3.642498
0     35.0    596.0             tripleo      12.681159       5.052132
0     89.0    577.0   Quality Assurance      14.037855       8.326118
0     52.0    572.0 

In [50]:
df_summary.sort_values("project", ascending=True)


,authors,commits,project,ratio_authors,ratio_commits
0,17.0,49.0,Chef OpenStack,19.540230,2.552083
0,119.0,2266.0,Documentation,21.915285,33.382440
0,1.0,18.0,I18n,7.692308,16.822430
0,145.0,2638.0,Infrastructure,11.798210,5.828289
0,8.0,637.0,OpenStack Charms,7.339450,3.642498
0,26.0,350.0,OpenStackAnsible,14.942529,2.972147
0,37.0,386.0,OpenStackClient,20.218579,21.035422
0,587.0,11950.0,Packaging-deb,14.745039,9.208600
0,2.0,28.0,Packaging-rpm,6.451613,2.994652
0,32.0,868.0,Puppet OpenStack,9.638554,10.886743


In [51]:
df_summary.columns
df_summary.columns = ["project", "authors", "commits", "ratio_authors", "ratio_commits"]
df_summary

,project,authors,commits,ratio_authors,ratio_commits
0,587.0,11950.0,Packaging-deb,14.745039,9.208600
0,145.0,2638.0,Infrastructure,11.798210,5.828289
0,149.0,4150.0,notavailable,16.779279,14.403221
0,8.0,637.0,OpenStack Charms,7.339450,3.642498
0,89.0,706.0,neutron,16.036036,6.961838
0,26.0,350.0,OpenStackAnsible,14.942529,2.972147
0,97.0,766.0,nova,16.193656,7.432564
0,119.0,2266.0,Documentation,21.915285,33.382440
0,35.0,596.0,tripleo,12.681159,5.052132
0,32.0,868.0,Puppet OpenStack,9.638554,10.886743


# Mailing Lists

In [28]:
INDEX = "mbox_openstack_gender_uuids_fielddata"


In [29]:
METRIC_NAME = "emails"
METRIC_FIELD = "id"

df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y])

In [30]:
draw_evolutionary_chart("key_as_string", METRIC_NAME)

In [31]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_4y])

draw_piechart(pie_chart_labels, pie_chart_values, 'Emails by Gender (last 4 years)')

In [32]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_1y])

draw_piechart(pie_chart_labels, pie_chart_values, 'Emails by Gender (last year)')

In [53]:
i18n_mailing_lists = ["http://lists.openstack.org/pipermail/openstack-ir/",
"http://lists.openstack.org/pipermail/openstack-de/",
"http://lists.openstack.org/pipermail/openstack-i18n-fr/",
"http://lists.openstack.org/pipermail/openstack-i18n/",
"http://lists.openstack.org/pipermail/openstack-el/",
"http://lists.openstack.org/pipermail/openstack-it/",
"http://lists.openstack.org/pipermail/openstack-zh/",
"http://lists.openstack.org/pipermail/openstack-es/",
"http://lists.openstack.org/pipermail/openstack-vi/",
"http://lists.openstack.org/pipermail/openstack-i18n-de/"]

dev_mailinglist = ["http://lists.openstack.org/pipermail/openstack-dev/"]

others_mls =["http://lists.openstack.org/pipermail/openstack/",
             "http://lists.openstack.org/pipermail/openstack-operators/",
             "http://lists.openstack.org/pipermail/openstack-docs/",
             "http://lists.openstack.org/pipermail/openstack-stable-maint/",
             "http://lists.openstack.org/pipermail/openstack-infra/",
             "http://lists.openstack.org/pipermail/openstack-security/",
             "http://lists.openstack.org/pipermail/openstack-announce/",
             "http://lists.openstack.org/pipermail/user-committee/",
             "http://lists.openstack.org/pipermail/openstack-announce/",
             "http://lists.openstack.org/pipermail/foundation/",
             "http://lists.openstack.org/pipermail/community/",
             "http://lists.openstack.org/pipermail/openstack-tc/",
             "http://lists.openstack.org/pipermail/women-of-openstack/",
             "http://lists.openstack.org/pipermail/openstrack-track-chairs/",
             "http://lists.openstack.org/pipermail/release-job-failures/",
             "http://lists.openstack.org/pipermail/enterprise-wg/",
             "http://lists.openstack.org/pipermail/legal-discuss/",
             "http://lists.openstack.org/pipermail/foundation-board/",
             "http://lists.openstack.org/pipermail/third-party-announce/",
             "http://lists.openstack.org/pipermail/openstack-personas/",
             "http://lists.openstack.org/pipermail/openstack-hpc/",
             "http://lists.openstack.org/pipermail/interop-wg/",
             "http://lists.openstack.org/pipermail/openstack-internships/",
             "http://lists.openstack.org/pipermail/openstack-sigs/",
             "http://lists.openstack.org/pipermail/elections-committer/",
             "http://lists.openstack.org/pipermail/transparency/",
             "http://lists.openstack.org/pipermail/foundation-testing-standards/",
             "http://lists.openstack.org/pipermail/openstack-content/",
             "http://lists.openstack.org/pipermail/openstack-api-consumer/",
             "http://lists.openstack.org/pipermail/openstack-sos/",
             "http://lists.openstack.org/pipermail/superuser/",
             "http://lists.openstack.org/pipermail/tax-affairs/"
]

filter_mls_languages = Q('terms', mailinglist__keyword=i18n_mailing_lists)
filter_mls_dev = Q('terms', mailinglist__keyword=dev_mailinglist)
filter_mls_others = Q('terms', mailinglist__keyword=others_mls)



In [45]:
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, filter_mls_dev])
draw_evolutionary_chart("key_as_string", METRIC_NAME)

In [48]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_4y, filter_mls_dev])

draw_piechart(pie_chart_labels, pie_chart_values, 'Openstack-Dev Emails by Gender (last 4 years)')

In [49]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_1y, filter_mls_dev])

draw_piechart(pie_chart_labels, pie_chart_values, 'OpenStack-Dev Emails by Gender (last year)')

In [50]:
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, filter_mls_languages])
draw_evolutionary_chart("key_as_string", METRIC_NAME)

In [51]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_4y, filter_mls_languages])

draw_piechart(pie_chart_labels, pie_chart_values, 'Lang-focused Emails by Gender (last 4 years)')

In [52]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_1y, filter_mls_languages])

draw_piechart(pie_chart_labels, pie_chart_values, 'Lang-focused Emails by Gender (last year)')

In [54]:
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, filter_mls_others])
draw_evolutionary_chart("key_as_string", METRIC_NAME)

In [55]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_4y, filter_mls_others])

draw_piechart(pie_chart_labels, pie_chart_values, 'Other MLS Emails by Gender (last 4 years)')

In [56]:
pie_chart_labels, pie_chart_values, pie_chart_count = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_1y, filter_mls_others])

draw_piechart(pie_chart_labels, pie_chart_values, 'Other MLS Emails by Gender (last year)')